# Projeto Aprendizado de Máquina

## CMC-13 Introdução a Ciência de Dados
## Equipe:
### André Luiz de Melo Thiessen
### Nikollas da Silva Antes
### Pedro Anacleto Martins Senna de Oliveira

In [53]:
#Importando as bibliotecas necessárias
import pandas as pd
from datetime import datetime
from datetime import date

#Importando as bases de dados
movies = pd.read_csv("movies.csv", on_bad_lines='skip', sep = ';')
ratings = pd.read_csv("ratings.csv", sep = ';')
users = pd.read_csv("users.csv")

# Verificando a existência de valores nulos (dados faltantes)
print(movies.isnull().values.any())
print(ratings.isnull().values.any())
print(users.isnull().values.any())

False
False
False


In [72]:
print (date.today())
display(users)

2022-04-17


,UserID,Gender,Occupation,Zip-code,name,birthday
0,1,F,10,48067,Isla Johnson,2012-01-04
1,2,M,16,70072,David Byrne,9/21/1964
2,3,M,15,55117,Jake Evans,4/3/1995
3,4,M,7,02460,Oliver Garcia,5/8/1974
4,5,M,20,55455,Liam Evans,6/18/1996
...,...,...,...,...,...,...
6035,6036,F,15,32603,Lauren Kelly,10/17/1995
6036,6037,F,1,76006,Olivia Walsh,6/12/1975
6037,6038,F,1,14706,Patricia Wilson,1/17/1963
6038,6039,F,0,01060,Margaret Miller,10/6/1977


In [71]:
#Como não há nenhum valor nulo em nenhum dos dataframes, podemos prosseguir

#Vamos transformar os dados de data de nascimento segundo a idade do usuário
print(users['birthday'][0])

#Função para calcular idade a partir de data
# def calculateAge(birthDate): 
#     today = date.today() 
#     age = today.year - birthDate.year
#     if (today.month, today.day) < (birthDate.month, birthDate.day):
#         age = age - 1
#     return age

# print(calculateAge(25/12/2010))

# for i in users.index:
#     print(users['birthday'][i])
#     if calculateAge(users['birthday'][i])<18:
#         users['birthday'][i]=1
#     print(calculateAge(users['birthday'][i]))


# print(users.info())

2012-01-04
